<a href="https://colab.research.google.com/github/Raymond-0314/ProgrammingAndApplication/blob/main/hw02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW02 Web Crawler

## 題目

台幣兌換外幣匯率查詢

## 摘要

本報告旨在透過網路爬蟲技術，收集 [比率網](https://www.findrate.tw/) 網站上的台幣兌換外幣資料，為旅遊和投資提供參考。

## 引言

在全球化的背景下，外幣兌換的需求不斷上升，無論是出行、商務還是投資，匯率的變動都會直接影響到成本和收益。比率網提供即時且多樣化的台幣對外幣的資訊，成為民眾的重要參考來源。然而，手動收集數據的效率較低，難以進行系統化的分析。本報告利用網路爬蟲技術，自動提取比率網的匯率數據，並進行整理和分析，以便為用戶提供更準確的決策支持。

## 方法

- 目標網站描述
  - 目標網站：[比率網銀行匯率查詢頁面](https://www.findrate.tw/exchange-quotation.php)。
  - 頁面結構：包含外幣種類、匯率、銀行和匯率更新時間等資訊。

- 工具與技術
  - 使用 Python 的 `requests` 庫發送 HTTP 請求。
  - 使用 `BeautifulSoup` 解析 HTML。
  - 使用 `google.colab` `google.auth` `gspread` 將數據儲存於 Google 試算表。


In [103]:
# 安裝函式庫
import requests
from bs4 import BeautifulSoup

from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user() # 進行授權
creds, _ = default() # 使用 default 方法獲取驗證信息
gc = gspread.authorize(creds) # 使用 creds 變數授權 gspread 模組

In [172]:
# 爬取資料程式碼
url = 'https://www.findrate.tw/exchange-quotation.php'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'lxml')
soup.title

# 取得所有銀行連結

bank_li = soup.find_all('li')
bank_list = []

for row in bank_li:
  a_tag = row.find('a')
  if a_tag:
    bank_url = 'https://www.findrate.tw/' + a_tag['href']
    bank_name = a_tag.text

    bank_list.append([bank_name, bank_url])
    #print(bank_name, bank_url)
#print(bank_list)


# 開始抓取所有銀行資料
data = [] #存取總資料

for bank in bank_list:
  bank_name = bank[0]
  bank_url = bank[1]

  response = requests.get(bank[1])
  soup = BeautifulSoup(response.text, 'lxml')
  #print(soup.title)

  #抓取匯率更新時間
  time = soup.find('a', id='rate_date').text.strip()

  #抓取該銀行匯率
  data_table = soup.find('tbody').find_all('tr')
  notFirst = False
  if len(data_table) > 1: # 有些銀行沒有顯示匯率 若無顯示匯率則跳過不抓取任何資料

    data_per_bank = []
    for row in data_table:
      if notFirst:
        #flag_url = row.find('img')['src']
        currency = row.find('td', class_="flag").text.strip()
        buy_in_cash = row.find_all('td')[1].text
        sell_out_cash = row.find_all('td')[2].text
        buy_in_deposit = row.find_all('td')[3].text
        sell_out_deposit = row.find_all('td')[4].text


        data_row = [currency, buy_in_cash, sell_out_cash, buy_in_deposit, sell_out_deposit]
        data_per_bank.append(data_row)
        #print(data_row)


      else:
        notFirst = True
    data.append([bank_name, time, data_per_bank])
  #print(bank_name)
    #print(data_per_bank)

print(data)


[['臺灣銀行', '2025-05-18 19:04', [['港幣 HKD', '3.705', '3.909', '3.826', '3.896'], ['澳幣 AUD', '18.93', '19.71', '19.145', '19.49'], ['新加坡幣 SGD', '22.64', '23.55', '23.11', '23.33'], ['日幣 JPY', '0.198', '0.2108', '0.2048', '0.2098'], ['馬來幣 MYR', '5.997', '7.522', '--', '--'], ['加拿大幣 CAD', '21.12', '22.03', '21.45', '21.78'], ['菲國比索 PHP', '0.4747', '0.6067', '--', '--'], ['印尼幣 IDR', '0.00148', '0.00218', '--', '--'], ['瑞士法郎 CHF', '35.2', '36.4', '35.81', '36.2'], ['南非幣 ZAR', '--', '--', '1.628', '1.718'], ['泰幣 THB', '0.7762', '0.9662', '0.8866', '0.9326'], ['歐元 EUR', '32.88', '34.22', '33.395', '33.995'], ['韓元 KRW', '0.01983', '0.02373', '--', '--'], ['人民幣 CNY', '4.089', '4.251', '4.156', '4.216'], ['美金 USD', '29.77', '30.44', '30.095', '30.245'], ['英鎊 GBP', '38.86', '40.98', '39.755', '40.385'], ['瑞典幣 SEK', '--', '--', '3.03', '3.15'], ['紐元 NZD', '17.27', '18.12', '17.6', '17.9'], ['越南盾 VND', '0.00092', '0.00133', '--', '--']]], ['三信商銀', '2025-05-18 19:04', [['澳幣 AUD', '--', '--', '19.27', 

## 結果


- 數據存儲方式
  - 將數據存儲於 [Google 試算表](https://docs.google.com/spreadsheets/d/16l54KYpxeDDs0Y54rddMT4DWi8gDViqJLoT3ngIiK-U/edit?gid=2028470349#gid=2028470349)，方便後續分析。

In [178]:
# 展示資料程式碼

# 開啟 Google 試算表
url = 'https://docs.google.com/spreadsheets/d/16l54KYpxeDDs0Y54rddMT4DWi8gDViqJLoT3ngIiK-U/edit?gid=0#gid=0'
workbook = gc.open_by_url(url)

# 重置工作表數量
if len(data) > len(workbook.worksheets()):
  for i in range(len(data) - len(workbook.worksheets())):
    workbook.add_worksheet(title="", rows="100", cols="20")
elif len(data) < len(workbook.worksheets()):
  for i in range(len(workbook.worksheets()) - len(data)):
    workbook.del_worksheet(workbook.worksheets()[-1])

# 重置工作表名稱
for i in range(len(data)):
  workbook.get_worksheet(i).update_title(str(i))


# 填入資料
sheet_num = len(workbook.worksheets())
i = 0
for data_row in data:
  # 工作表數量不足就新增
  if i + 1 > sheet_num:
    workbook.add_worksheet(title="", rows="100", cols="20")

  #print(data_row[0])
  # 取得工作表
  sheet = workbook.get_worksheet(i)
  sheet.clear()
  # 修改工作表名稱 新增匯率更新時間
  sheet.update_title(data_row[0])
  sheet.append_row([data_row[0] + ' (匯率更新時間 : ' + data_row[1] + ')'])
  sheet.merge_cells('A1:E1')
  # 新增標題列
  sheet.append_row(['幣別','現鈔買入','現鈔賣出','即期買入','即期賣出'])
  for data_per_currency in data_row[2]:
    sheet.append_row(data_per_currency)

  i += 1

## 問題與挑戰

- 技術挑戰  
    各銀行匯率顯示於不同頁面，須先取得各銀行連結後，使用迴圈透過連結前進各頁面分別抓取匯率資料。
- 資料限制  
    部分銀行無顯示匯率資料，需跳過。

## 結論


- 成功獲取並儲存各銀行匯率資料。
- 目前僅抓取台幣兌換外幣的資料，未來可擴展至外幣兌換台幣、外幣兌換外幣等資訊。

## 參考文獻

* [比率網](https://www.findrate.tw/)